# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [5]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [6]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {    
    "journal":{
        "file": "pubs.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    }
}

In [7]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [8]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields
        
        try:
            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = bib_id + ".md" # (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = bib_id + ".html" # (str(pub_date) + "-" + url_slug).replace("--","-")

            #Build Citation from text
            citation = ""

            #citation authors - todo - add highlighting for primary author?
            for author in bibdata.entries[bib_id].persons["author"]:
                citation = citation+" "+author.first_names[0]+" "+author.last_names[0]+", "

            #citation title
            citation = citation + "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""

            #add venue logic depending on citation type
            
            venue = ""
            if "journal" in b.keys():
                venue = publist[pubsource]["venue-pretext"]+b["journal"].replace("{", "").replace("}","").replace("\\","")
            elif "booktitle" in b.keys():
                venue = publist[pubsource]["venue-pretext"]+b["booktitle"].replace("{", "").replace("}","").replace("\\","")
            else:
                venue = ""
            citation = citation + " " + html_escape(venue)
            citation = citation + ", " + pub_year + "."

            
            ## YAML variables
            md = "---\ntitle: \""   + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + '"\n'
            
            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            if bibdata.entries[bib_id].type in ["phdthesis"]:
                md += """\ncategory: books"""
            else:
                md += """\ncategory: manuscripts"""
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            if len(venue) > 5:
                md += "\nvenue: '" + html_escape(venue) + "'"
            
            url = False
            urlhref = "../files/" + bib_id + ".pdf"
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    #md += "\npaperurl: '" + b["url"] + "'"
                    urlhref = b["url"]
                    url = True
#                else:
#                    url = False
#            else:
#                md += "\npaperurl: '" + "../files/" + bib_id + ".pdf" + "'"
#                url = True
            # md += "\ncitation: '" + html_escape(citation) + "'"

            md += "\n---"

            
            ## Markdown description for individual page

            md += "\n## Authors\n"
            authors = [
                author.first_names[0] + " " + author.last_names[0]
                for author in bibdata.entries[bib_id].persons["author"]
            ]
            md += ", ".join(authors)

            if(b["abstract"]): 
                md += "\n## Abstract\n"
                md += b["abstract"]
            
            if note:
                md += "\n" + html_escape(b["note"]) + "\n"

            md += "\n## Downloads\n"

            preprintFileName = "../files/" + bib_id + ".pdf"
            if os.path.exists(preprintFileName):
                md += "\n[Preprint](" + preprintFileName + "){:target=\"_blank\"}\n"             

            if url:
                md += "\n[Official Version](" + urlhref + "){:target=\"_blank\"}\n" 
            #else:
            

            md_filename = os.path.basename(md_filename)
            md += "\n[Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+"){:target=\"_blank\"}"

            with open("../_publications/" + md_filename, 'w', encoding="utf-8") as f:
                f.write(md)
            print(f'SUCESSFULLY PARSED {md_filename} {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


SUCESSFULLY PARSED Kuth25RTG.md Kuth25RTG: " {Real-Time GPU Tree Generation}  "
SUCESSFULLY PARSED Cigolle14SER.md Cigolle14SER: " A Survey of Efficient Representations for Independent Unit V ... "
SUCESSFULLY PARSED Meyer10OFN.md Meyer10OFN: " On Floating-Point Normal Vectors  "
SUCESSFULLY PARSED Suessmuth10SRB.md Suessmuth10SRB: " {Surface Reconstruction Based on Hierarchical Floating Radia ... "
SUCESSFULLY PARSED Schoenfeld103SC.md Schoenfeld103SC: " 3-SAT on CUDA: Towards a Massively Parallel SAT Solver  "
SUCESSFULLY PARSED Eisenacher09RVR.md Eisenacher09RVR: " Real-time View-dependent Rendering of Parametric Surfaces  "
SUCESSFULLY PARSED Selgrad15FMS.md Selgrad15FMS: " Filtering Multi-Layer Shadow Maps for Accurate Soft Shadows  "
SUCESSFULLY PARSED Schaefer13MAH.md Schaefer13MAH: " Multiresolution Attributes for Hardware Tessellated Objects  "
SUCESSFULLY PARSED Meyer09DPH.md Meyer09DPH: " {Data-Parallel Hierarchical Link Creation for Radiosity}  "
SUCESSFULLY PARSED Schaefer